In [ ]:
con = duckdb.connect()
train_sample = con.query(f"""
(
select * from parquet_scan('/home/jovyan/train.parquet')
where binds = 0
order by random()
limit 30000)
union all 
(select * from parquet_scan('/home/jovyan/train.parquet')
where binds = 1
order by random()
limit 30000
)""").df()
con.close()



In [ ]:
def generate_ecfp(molecule, radius=2, bits = 1024):
    if molecule is None:
        return None
    return list(AllChem.GetMorganFingerprintAsBitVect(molecule, radius, nBits=bits))


ecfp_udf = udf(generate_ecfp, StringType())

get_molecule_udf = udf(lambda x: Chem.MolFromSmiles(x), StringType())

In [ ]:
test_sample_data = sample_data.withColumn("molecule", get_molecule_udf(sample_data["molecule_smiles"].astype(StringType())))
# test_sample_data = test_sample_data.withColumn('ecfp', ecfp_udf(test_sample_data["molecule"]))
# test_sample_data.show()

In [ ]:
### Vector embedding molecule
train_sample['molecule'] = pandas_train_1['molecule_smiles'].apply(Chem.MolFromSmiles)
train_sample['ecfp'] = train_sample['molecule'].apply(generate_ecfp)


# ### Vector Embedding / Encoding protein name

oneHot = OneHotEncoder(sparse=False)
protein_oneHot = oneHot.fit_transform(train_sample['protein_name'].values.reshape(-1,1))

train_sample['encoded_protein_name'] = protein_oneHot.tolist()

# X = [ecfp + protein for ecfp, protein in zip(train_sample['ecfp'], train_sample['encoded_protein_name'])]
# y = train_sample['binds'].tolist()

# x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

# rf = RandomForestClassifier(n_estimators=100, random_state=42)
# rf.fit(x_train, y_train)

In [ ]:
predictions = rf.predict(x_test)
prob_predictions = rf.predict_proba(x_test)[:, 1]
print("Rounded Predictions:", average_precision_score(y_test, predictions))
print("Probability Predictions:", average_precision_score(y_test, prob_predictions))

# Test

In [ ]:
for test_csv in pd.read_csv("test.csv", chunksize=100_000):
    ### Vectorize molecule_smiles
    test_csv['molecule'] = test_csv['molecule_smiles'].apply(Chem.MolFromSmiles)
    test_csv['ecfp'] = test_csv['molecule'].apply(generate_ecfp)
    
    ### Encode protein_name
    oneHotEncoderTest = OneHotEncoder(sparse=False)
    test_csv['encoded_protein_name'] = oneHotEncoderTest.transform(test_csv['protein_name'].values.reshape(-1,1)).tolist()

    x_test = [ecfp + protein for ecfp, protein in zip(test_csv['ecfp'], test_csv['encoded_protein_name'])]
    y_test = test_csv['binds'].tolist()

    test_pred = rf.transform(x_test)
    accuracy_precision_score(y_test, test_pred)